In [1]:
import os
import math
import numpy as np
import cv2 as cv

from matplotlib import image as img  
from matplotlib import pyplot as plt 
from matplotlib.patches import Circle

import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
from explode import explode
#from pyproj import Proj

folder = '/Users/Zack/0_bing/detections_1+ positive/'
#folder = '/Users/Zack/0_bing/detections_1+/detections/' #yolo v3 predictions
#folder = '/Users/Zack/0_bing/images_1km_2019-10-13/' #control data
#folder = '/Users/Zack/0_bing/images_1km_2020-02-08/'#all SAR predictions

#out_folder = '/Users/Zack/Desktop/temp/'

images = os.listdir(folder)
print("total number of images:", len(images))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/Zack/0_bing/detections_1+ positive/'

In [ ]:
#hough parameters
minDist=20
param1=45
param2=33 #33
minRadius=10
maxRadius=30

#perform detection
c_list = []
for image in images: 
    img = cv.imread(cv.samples.findFile(folder + image), cv.IMREAD_COLOR) #load image
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) #grayscale
    img = cv.blur(img,(5,5)) #blur
    circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, 1, minDist=minDist, 
                              param1=param1, param2=param2, 
                              minRadius=minRadius, maxRadius=maxRadius)
    c_list.append(circles)

#images with detections - remove None
index = pd.DataFrame(c_list)
index['index'] = index.index
print(len(c_list), 'images')
print(index.index.tolist())

index = index.dropna()
index = index.index.tolist()
print(len(index), 'images with detections')
print(index)

#detection for images with detections
object_list = pd.DataFrame([])
for h in index:
    image = images[h]
    img = cv.imread(cv.samples.findFile(folder + image), cv.IMREAD_COLOR) #load image
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) #grayscale
    img = cv.blur(img,(5,5))
    #img = cv.medianBlur(img, 5)
    #img = cv.GaussianBlur(img, (7, 7), 8)

    circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, 1, minDist=minDist, 
                              param1=param1, param2=param2, 
                              minRadius=minRadius, maxRadius=maxRadius)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))   
        c, r = [], []
        for j in circles[0, :]:
            center = (j[0], j[1])
            radius = j[2]
            cv.circle(img, center, radius, (0, 0, 255), 3)
            c.append(center)
            r.append(radius) 
       
    df = pd.DataFrame(data = {'radius':r, 'center':c})  
    df['image_name'] = image #add image name

    #plot detections to image
    img = plt.imread(folder + image)
    fig, ax = plt.subplots(ncols = 1, nrows= 1, figsize = (20, 20))
    ax.axis('off')
    ax.imshow(img)

    for i in (range(len(df))):
        circle = Circle((c[i][0], c[i][1]), r[i], facecolor = 'none', edgecolor = '#32CD32')
        ax.add_patch(circle)

    #export images with annotations
    #plt.savefig(out_folder + image, dpi = 96, bbox_inches='tight', pad_inches=0)
    #plt.show()
    
    print(len(df), 'detections', image)
    object_list = object_list.append(pd.DataFrame(df), sort=False) 
    object_list.head()

In [ ]:
#select detections
count = pd.DataFrame(object_list['image_name'].value_counts())

#select images with 5 or more detections
select = count[count.image_name > 4]
select = select.index.tolist()
selected = object_list[object_list.image_name.isin(select)]
print(len(select), ' images with 5 or more detections')
print(len(object_list), 'total detections')
print(len(selected), 'selected detections')

#export csv
object_list.to_csv('detections_hough.csv', index = None, header=True)

In [ ]:
# selected = selected[0:500]
# selected.head()

In [ ]:
#georeference detections
lon_list, lat_list, res_list = [], [], []
for i in (range(len(selected))):
    
    #extract bottom right coordinate from image name
    name = selected['image_name'].iloc[i]
    br_lon = float(name.split(", ")[4]) #lon = x                                       
    br_lat = float(name.split(", ")[3]) #lat = y
    
    #extract top left coordinate from image name
    tl_lon = float((name.split(",")[6]).split(")")[0]) #lon = x
    tl_lat = float(name.split(", ")[5]) #lat = y
    
    #image dimensions
    image = os.path.join(folder, object_list['image_name'].iloc[i])
    dim = plt.imread(image).shape
    width = abs(br_lon - tl_lon) 
    height = abs(tl_lat - br_lat)
    
    #decimal degrees per pixel
    res_x = width / dim[0] #image width
    res_y = height / dim[1] #image height
    
    #res
    lat = tl_lat * math.pi / 180
    res = 156543.04 * math.cos(lat) / (2 ** 18)
    
    #convert centroid point to lat/lon  
    center = selected['center'].iloc[i]  
    x_center = tl_lon + (center[0] * res_x) 
    y_center = tl_lat - (center[1] * res_y) 

    lon_list.append(x_center)
    lat_list.append(y_center)
    res_list.append(res)

In [ ]:
#prepare df for export as shapefile
d2 = selected.reset_index(drop=True) #super important
d2 = d2.drop(columns=['center'])
d2['resolution']=res_list
d2['diameter']=d2['radius'] * res_list * 2
d2['radius']=d2['radius'] * res_list 

#add geometry information to df 
geometry = [Point(i) for i in zip(lon_list, lat_list)]
d2_centroids = gpd.GeoDataFrame(d2, geometry=geometry, crs={'init' :'EPSG:4326'})

#export centroids
d2_centroids.to_file('../targets/detections_hough_pts.geojson', driver='GeoJSON')
d2_centroids.to_file('../targets/detections_hough_pts.shp')

In [ ]:
#create buffer by diameter
def buffer(row):
     return row.geometry.buffer(row.radius)    
d2_centroids = d2_centroids.to_crs({'init': 'EPSG:2100'})
buffer = d2_centroids['geometry'] = d2_centroids.apply(buffer, axis=1)

d2_circles = gpd.GeoDataFrame(d2_centroids, geometry = buffer, crs={'init': 'EPSG:2100'})
d2_circles = d2_circles.to_crs({'init': 'EPSG:4326'})

#export polygons
d2_circles.to_file('../targets/detections_hough.geojson', driver='GeoJSON')
d2_circles.to_file('../targets/detections_hough.shp')

In [ ]:
#detection aggregation
#buffer points
buffer = gpd.read_file('../targets/detections_hough_5m.shp')
buffer = buffer.to_crs({'init': 'EPSG:4326'})

In [ ]:
#dissolve buffers
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')

#explode polygon 
buffer_exploded = explode(buffer_dis)  

In [ ]:
#generate mean diameter for each aggregated point
d_list = []
for i in (range(len(buffer_exploded))):
    buffer = buffer_exploded.iloc[i]
    d2_centroids.within(buffer) 
    subset = points[points.within(i)]
    diameter = np.mean(subset['diameter'])
    d_list.append(diameter)

In [ ]:
#generate centroids
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid, crs={'init': 'EPSG:2100'})

centroids.crs = {'init' :'ESPG:4326'}
#centroids = centroids.to_crs({'init': 'EPSG:4326'})
centroids.to_file('/Users/Zack/Desktop/clean.shp')

#generate new buffers for aggregated points using mean diameter
centroids
d2_centroids_buffer = d2_centroids['geometry'] = d2_centroids.apply(buffer, axis=1)
buffer = d2_centroids['geometry'] = d2_centroids.apply(buffer, axis=1)